# Ranking prototype (BM25 + Embeddings)

In [ ]:
import numpy as np
import string
from rank_bm25 import BM25Okapi
from pathlib import Path
from nltk.stem import PorterStemmer
from sentence_transformers import SentenceTransformer

STOPWORDS_FILE_PATH = Path.cwd().resolve() / "stopwords.txt"


In [ ]:
# Sample dataset of 10 creators
# dict where key = creator niche, value = creator bio

sample_dataset = {
    "fashion_minimalist_style": """I post daily outfit inspirations, seasonal lookbooks, and minimalist fashion tips focused on affordable pieces. 
My recent content includes capsule wardrobe guides, styling basics, and color-coordination breakdowns. 
Instagram posts feature outfit photos, thrifted finds, and short styling reels.
""",

"learning_to_code_journey": """I document my journey learning to code from scratch, posting study routines, project logs, and beginner-friendly explanations. 
Recent videos show progress building Python scripts, simple web apps, and solving DSA problems as a newcomer to tech. 
Instagram posts include accountability updates, motivational captions, and resources for absolute beginners.
""",

"backend_go_engineer": """I share backend engineering tutorials in Go, focusing on concurrency, networking, microservice design, and Docker-based workflows. 
My recent content includes deep dives into goroutines, REST API design, production logging, and containerizing real-world services. 
Instagram posts feature short Go tips, dev memes, and workflow optimizations for backend developers.
""",

"fitness_home_workouts": """I create daily home workout routines targeting fat loss, mobility, and functional strength using bodyweight or minimal equipment. 
My latest videos include 10-minute HIIT sessions, beginner full-body circuits, and nutrition tips for sustainable weight loss. 
Instagram posts share motivational progress photos, short workout reels, and simple healthy meal ideas.
""",

"beauty_skincare_reviewer": """I review skincare products and routines for acne-prone, oily, and sensitive skin types, focusing on ingredient science. 
Recent videos compare retinol serums, sunscreen textures, exfoliants, and Korean skincare routines. 
Instagram posts share product flatlays, morning/night routines, and short ingredient breakdowns.
""",

"personal_finance_educator": """I simplify personal finance topics such as index investing, budgeting systems, emergency funds, and tax optimization. 
My recent content includes ETF comparisons, beginner investment strategies, and monthly market breakdowns. 
Instagram posts include budgeting templates, money habits, and short explainers on compounding and inflation.
""",

"healthy_cooking_mealprep": """I share easy, healthy recipes and weekly meal prep ideas designed for busy students and working professionals. 
My latest videos feature quick high-protein dinners, 15-minute lunches, and budget-friendly vegetarian meal preps. 
Instagram posts include grocery hauls, step-by-step meal reels, and simple nutrition tips.
""",

"yoga_mindfulness_coach": """I guide yoga flows, stretching routines, and mindfulness practices aimed at improving mobility, posture, and mental clarity. 
Recent videos include morning yoga sessions, hip-opening flows, and breathwork techniques for stress relief. 
Instagram posts share short mobility drills, inspirational quotes, and meditation reminders.
""",

"cloud_devops_engineer": """I break down cloud engineering concepts including AWS, Kubernetes, Docker, and CI/CD automation. 
My latest tutorials explain Terraform modules, EKS deployments, load balancing, and building secure production pipelines. 
Instagram posts include cloud diagrams, quick DevOps tips, and workflow comparisons for SREs and platform engineers.
""",

"frontend_react_engineer": """I teach React, TypeScript, and modern frontend engineering with a focus on clean UI patterns and reusable components. 
Recent videos cover state management, hooks, responsive layouts, and building production-ready interfaces with React and Tailwind. 
On Instagram, I post quick JavaScript tips, UI design breakdowns, and small project showcases.
"""
}

In [ ]:
# Keyword search implementation

class KeywordSearch:
    def __init__(self, documents: dict = sample_dataset):
        self.documents = documents
        self.index = None

        # list of lists, each list tokenized creator 
        self.tokenized_corpus = []

        # save creator niches in same order in which added to corpus
        self.ordered_creators = []

    def create_bm25_index(self):
        tokenized_creators = []
        creators_ordered = []
        for creator, bio in self.documents.items():
            tokenized_creator = process_text_to_tokens(f"{creator}: {bio}")
            tokenized_creators.append(tokenized_creator)
            creators_ordered.append(creator)

        self.tokenized_corpus = tokenized_creators
        self.ordered_creators = creators_ordered

        self.index = BM25Okapi(tokenized_creators)
    
    def bm25_search(self, query: str):
        if self.index is None:
            self.create_bm25_index()

        tokenized_query = process_text_to_tokens(query)

        scores = self.index.get_scores(query=tokenized_query)

        results = []
        for i, score in enumerate(scores):
            creator = self.ordered_creators[i]
            results.append((creator, score))

        return results

        
        


def remove_all_punctuation_lowercase(text: str) -> str:
    tt = str.maketrans("", "", string.punctuation)
    return text.translate(tt).lower()

def tokenize(text: str) -> list[str]:
    return text.lower().split()

def remove_stop_words(tokens: list[str]) -> list[str]:
    with open(STOPWORDS_FILE_PATH, "r") as f:
        stop_words = f.readlines()
    
    result = []
    for token in tokens:
        if token in stop_words:
            continue
        result.append(token)

    return result

def stem_tokens(tokens: list[str]) -> list[str]:
    stemmer = PorterStemmer()
    return list(map(lambda token: stemmer.stem(token), tokens))

def process_text_to_tokens(text: str) -> list[str]:
    tokens = remove_all_punctuation_lowercase(text=text)
    tokens = tokenize(tokens)
    tokens = remove_stop_words(tokens)
    tokens = stem_tokens(tokens)
    return tokens